<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/maestro_alpha_trend_signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kütüphaneler
!pip install git+https://github.com/rongardF/tvdatafeed backtesting
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="backtesting")

# Zaman dilimi ismini döndürme fonksiyonu
def interval_name(interval):
    interval_map = {
        Interval.in_15_minute: '15 Dakika',
        Interval.in_30_minute: '30 Dakika',
        Interval.in_45_minute: '45 Dakika',
        Interval.in_1_hour: '1 Saat',
        Interval.in_2_hour: '2 Saat',
        Interval.in_4_hour: '4 Saat',
        Interval.in_daily: 'Günlük',
        Interval.in_weekly: 'Haftalık',
        Interval.in_monthly: 'Aylık'
    }
    return interval_map.get(interval, 'Günlük')

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimini seçin:")
    for key, (name, _) in intervals.items():
        print(f"{key}. {name}")

    choice = input("Seçiminiz (örnek: 6): ").strip()
    return intervals.get(choice, ('Günlük', Interval.in_daily))

# Standart Moving Average
def sma(series, length):
    return series.rolling(window=length).mean()

def mfi(high, low, close, volume, window=14):
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    positive_money_flow = (money_flow * (close > close.shift(1))).rolling(window=window).sum()
    negative_money_flow = (money_flow * (close < close.shift(1))).rolling(window=window).sum()
    money_ratio = positive_money_flow / negative_money_flow
    mfi = 100 - (100 / (1 + money_ratio))
    return mfi

def tr(high, low, close):
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr

def Alpha_Trend(df, mult=1, n1=14):
    df['TR'] = tr(df['High'], df['Low'], df['Close'])
    df['ATR'] = sma(df['TR'], length=n1)
    df['mfi'] = mfi(df['High'], df['Low'], df['Close'], df['Volume'], n1)
    df['upT'] = df['Low'] - df['ATR'] * mult
    df['downT'] = df['High'] + df['ATR'] * mult
    df['AlphaTrend'] = 0.0
    alpha_trend_values = [0.0]

    for i in range(1, len(df)):
        if df['mfi'].iloc[i] >= 50:
            alpha_trend_values.append(max(df['upT'].iloc[i], alpha_trend_values[i-1]))
        else:
            alpha_trend_values.append(min(df['downT'].iloc[i], alpha_trend_values[i-1]))

    df['AlphaTrend'] = alpha_trend_values
    df['Entry'] = False
    prev_signal = False
    for i in range(2, len(df)):
        if df.loc[i, 'AlphaTrend'] > df.loc[i-2, 'AlphaTrend']:
            df.loc[i, 'Entry'] = True
            prev_signal = True
        elif df.loc[i, 'AlphaTrend'] == df.loc[i-2, 'AlphaTrend'] and prev_signal:
            df.loc[i, 'Entry'] = True
        else:
            prev_signal = False
    df['Exit'] = df['Entry'] == False
    return df

# Kullanıcıdan zaman dilimi al
selected_name, selected_interval = get_interval_choice()
print(f"Seçilen Zaman Dilimi: {selected_name}")

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Raporlama için kullanılacak başlıklar
Titles = ['Hisse Adı', 'Son Fiyat', 'Kazanma Oranı', 'Giriş Sinyali', 'Çıkış Sinyali']

df_signals = pd.DataFrame(columns=Titles)

# Backtest için gerekli class yapısı
from backtesting import Backtest, Strategy

class MyStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()
        elif self.data['Exit'] == True:
            self.position.close()

# Hisse verilerini çekme ve analiz etme
for i in range(0, len(Hisseler)):
    try:
        mult = 1
        n1 = 14
        # Zaman dilimi parametresini seçilen interval ile değiştir
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=selected_interval, n_bars=500)
        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        data = data.reset_index()
        AlphaTrend = Alpha_Trend(data, mult, n1)
        AlphaTrend['datetime'] = pd.to_datetime(AlphaTrend['datetime'])
        AlphaTrend.set_index('datetime', inplace=True)
        bt = Backtest(AlphaTrend, MyStrategy, cash=100000, commission=0.002)
        Stats = bt.run()
        Buy = False
        Sell = False
        Signals = AlphaTrend.tail(2)
        Signals = Signals.reset_index()
        Buy = Signals.loc[0, 'Entry'] == False and Signals.loc[1, 'Entry'] == True
        Sell = Signals.loc[0, 'Exit'] == False and Signals.loc[1, 'Exit'] == True
        Last_Price = Signals.loc[1, 'Close']
        L1 = [Hisseler[i], Last_Price, round(Stats.loc['Win Rate [%]'], 2), str(Buy), str(Sell)]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except:
        pass

# Giriş sinyali olan hisse senetlerini yazdır
df_True = df_signals[(df_signals['Giriş Sinyali'] == 'True')]
print(df_True.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-63aq_wk1
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-63aq_wk1
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=e4067467ab056ad532c51dcf413b9f0e4538917e91e1e3458a4939285016d7c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-6livj5t1/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=dc0c25ddc13ad8bdb49dc5b17c3751341ee6440535aa79d5db2ddbb5fcf96559
  Stored in directory: /root/.cache/pip/wheels/2c/56/19/bf7ee5e164aa99a976e3f64841c83b5a

Seçilen Zaman Dilimi: Günlük
['A1CAP', 24.84, 20.0, 'False', 'False']
['ACSEL', 118.0, 28.57, 'False', 'False']
['ADEL', 33.68, 28.57, 'False', 'False']
['ADESE', 2.26, 37.5, 'False', 'False']
['ADGYO', 31.02, 20.0, 'False', 'False']
['AEFES', 165.7, 80.0, 'False', 'False']
['AFYON', 15.18, 55.56, 'False', 'False']
['AGESA', 141.9, 60.0, 'False', 'False']
['AGHOL', 309.0, 80.0, 'False', 'False']
['AGROT', 10.34, 50.0, 'False', 'False']
['AGYO', 7.19, 25.0, 'False', 'False']
['AHGAZ', 19.93, 50.0, 'False', 'False']
['AHSGY', 23.56, 0.0, 'False', 'False']
['AKBNK', 66.0, 33.33, 'False', 'False']
['AKCNS', 184.6, 42.86, 'False', 'False']
['AKENR', 14.38, 28.57, 'False', 'False']
['AKFGY', 2.12, 42.86, 'False', 'False']
['AKFYE', 19.77, 50.0, 'False', 'False']
['AKGRT', 7.99, 50.0, 'False', 'False']
['AKMGY', 219.3, 60.0, 'False', 'False']
['AKSA', 11.07, 33.33, 'False', 'False']
['AKSEN', 40.68, 57.14, 'False', 'False']
['AKSGY', 7.42, 71.43, 'False', 'False']
['AKSUE', 11.99, 22.22, 'Fal

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['YYLGD', 9.86, 66.67, 'False', 'False']
['Z30KE', 97.72, 42.86, 'False', 'False']
['Z30KP', 138.95, 83.33, 'False', 'False']
['ZEDUR', 8.6, 40.0, 'False', 'False']
['ZGOLD', 325.5, 100.0, 'False', 'False']
['ZOREN', 4.23, 28.57, 'False', 'False']
['ZPBDL', 151.4, 42.86, 'False', 'False']
['ZPLIB', 172.1, 50.0, 'False', 'False']
['ZRGYO', 13.74, 66.67, 'False', 'False']
    Hisse Adı  Son Fiyat  Kazanma Oranı Giriş Sinyali Çıkış Sinyali
281     ISBTR  529502.50          25.00          True         False
332     KRGYO       9.69          60.00          True         False
368     MEGAP       3.10          66.67          True         False
549     VAKBN      25.50          33.33          True         False
